In [1]:
import cv2 as cv
import argparse
import sys
import numpy as np
import os.path


In [3]:
confThreshold = 0.5  # Confidence threshold
nmsThreshold = 0.4  # Non-maximum suppression threshold

inpWidth = 416  # 608     # Width of network's input image
inpHeight = 416  # 608     # Height of network's input image

In [5]:
classesFile = "classes.names"

classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

In [21]:
modelConfiguration = "darknet-yolov3.cfg"
modelWeights = "model.weights"

net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [22]:
def getOutputsNames(net):
    layersNames = net.getLayerNames()
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

def drawPred(classId, conf, left, top, right, bottom):
    cv.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)

    label = '%.2f' % conf
    if classes:
        assert(classId < len(classes))
        label = '%s: %s' % (classes[classId], label)

    labelSize, baseLine = cv.getTextSize(
        label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(
        1.5*labelSize[0]), top + baseLine), (255, 0, 255), cv.FILLED)
    cv.putText(frame, label, (left, top),
               cv.FONT_HERSHEY_SIMPLEX, 0.70, (255, 255, 255), 2)

def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    classIds = []
    confidences = []
    boxes = []

    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        print("out.shape : ", out.shape)
        for detection in out:

            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if detection[4] > confThreshold:
                print(detection[4], " - ", scores[classId], " - th : ", confThreshold)
                print(detection)
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(classIds[i], confidences[i], left,
                 top, left + width, top + height)


In [26]:
dataset = "data/" 
for filename in os.listdir(dataset):
    frame = cv.imread(os.path.join(dataset, filename))
    blob = cv.dnn.blobFromImage(
        frame, 1/255, (inpWidth, inpHeight), [0, 0, 0], 1, crop=False)

    net.setInput(blob)
    outs = net.forward(getOutputsNames(net))

    postprocess(frame, outs)

    t, _ = net.getPerfProfile()
    label = 'Inference time: %.2f ms' % (t * 1000.0 / cv.getTickFrequency())
    cv.putText(frame, label, (0, 15), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))

    cv.imshow("out", frame)
    cv.waitKey(10)
cv.destroyAllWindows()

out.shape :  (507, 6)
out.shape :  (2028, 6)
0.99768853  -  0.9974393  - th :  0.5
[0.49053425 0.6198072  0.14297055 0.07790624 0.99768853 0.9974393 ]
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
0.9999808  -  0.99992895  - th :  0.5
[0.5670549  0.5225068  0.13970521 0.0907565  0.9999808  0.99992895]
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
0.5418358  -  0.0  - th :  0.5
[0.49939364 0.5969842  0.24669178 0.10686862 0.5418358  0.        ]
0.9998715  -  0.9997114  - th :  0.5
[0.5094192  0.59727365 0.24846333 0.09600782 0.9998715  0.9997114 ]
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
0.999979  -  0.99985087  - th :  0.5
[0.51163787 0.48957017 0.23886178 0.05215568 0.999979   0.99985087]
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
0.55959773  -  0.55843824  - th :  0.5
[0.5443837  0.8579858  0.24285422 0.08817552 0.55959773 0.55843824]
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape 